<a href="https://colab.research.google.com/github/btg1631/study_data_analytics/blob/main/docs/quests/CDAs/CDA_continuous.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 한글 폰트 설치 및 시각화

In [ ]:
# 한글 폰트 설치
!apt-get install -y fonts-nanum
!fc-cache -fv
!rm ~/.cache/matplotlib -rf
# 설치 후 colab 경우 Reuntime > Restart session 필요

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 32 not upgraded.
Need to get 10.3 MB of archives.
After this operation, 34.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-nanum all 20200506-1 [10.3 MB]
Fetched 10.3 MB in 1s (11.0 MB/s)
Selecting previously unselected package fonts-nanum.
(Reading database ... 121730 files and directories currently installed.)
Preparing to unpack .../fonts-nanum_20200506-1_all.deb ...
Unpacking fonts-nanum (20200506-1) ...
Setting up fonts-nanum (20200506-1) ...
Processing triggers for fontconfig (2.13.1-4.2ubuntu5) ...
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/

In [2]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

mpl.rc('font', family='NanumBarunGothic') # 혹은 다른 설치한 Nanum 폰트 사용

## load dataset

In [5]:
import pandas as pd
df_FRUITS = pd.read_csv('fruitstore_saleshistory.csv')
df_FRUITS[:2]

,공급일자,요일,공급월,공급주차,회원번호,조합원상태,물품대분류,물품중분류,물품소분류,물품명,구매수량,주소-구,주소-동,성별,연령,연령대,구매금액,구매매장,반품_원거래일자,구매시각
0,2018-01-02,화,1.0,1.0,2.723699e+08,정상회원,과실,과일,사과,사과/유(1.5kg),1.0,수지구,풍덕천동,여,45.0,40대,22207.0,매장C,NaN,10:04
1,2018-01-02,화,1.0,1.0,1.506656e+09,정상회원,채소,버섯,느타리버섯,느타리버섯(300g),1.0,수지구,풍덕천동,여,36.0,30대이하,4977.0,매장C,NaN,10:05


## X(연속형) / Y(연속형) 케이스

- 판매된 수량(X)이 각 과일 유형의 단가(Y)와 어떤 관계가 있습니까?
 + 분석 방법: 수량과 가격 사이의 관계의 강도와 방향을 측정하기 위한 피어슨 또는 스피어만


- 귀무 가설 : 수량이 과일 유형의 단가와 관계가 없다.
- 대립 가설 : 수량이 과일 유형의 단가와 관계가 있다.

In [ ]:
fruits = '물품중분류 == "과일"'
df_FRUITS_fruits = df_FRUITS.query(fruits)
df_FRUITS_fruits['물품명'].unique()
# '사과', '건대추', '귤(조생)', '배', '산딸기', '단감', '참다래', '레몬', '블루베리', '곶감',
# '귤만감', '냉동건조과일', '홍시용감(대봉)', '귤(비가림)

array(['사과/유(1.5kg)', '건대추(500g)', '귤/유(1.5kg)', '배(대:3개)', '사과/(2kg)/대',
       '냉동산딸기(250g)', '배(중:3개)', '단감/유(중:5개)', '참다래(골드:800g)', '귤/무(3kg)',
       '사과/(5kg)/중', '레몬/유(500g)', '사과/(2kg)/중', '귤/무(5kg)',
       '레몬/유,무(소:500g)', '귤/유(3kg)', '냉동블루베리(500g)', '곶감/고종시(유:12개)',
       '감귤/진홍(무:1.5kg)', '사과/무(1.5kg)', '단감/무(중:5개)', '귤/유(5kg)',
       '배/유(대:3개)', '단감/유(대:5개)', '사과/(2kg)/소', '대봉감말랭이(200g)',
       '바삭한대추(유/무:50g)', '홍시용감/대봉(무:3kg)', '참다래(그린:800g)', '레몬/무(500g)',
       '배/(대:5kg)', '감말랭이(300g)', '사과/(5kg)/대', '곶감/반건시(밀시감:500g)',
       '참다래(그린:2kg)', '참다래(골드:2kg)', '배/유(중:3개)', '황금향(무:2kg)',
       '곶감/둥시(10개)', '단감(특대:5개)', '곶감/둥시(30개)', '홍시용감/대봉(3kg)',
       '건대추/자주인증(500g)', '레드향/유(1.5kg)', '배(특대:1개)', '단감(대:5개)',
       '단감(중:5개)', '곶감/대봉/유기(10개)', '귤/무(1.5kg)', '감귤/비가림/유(1.5kg)',
       '감귤/비가림(유:3kg)', '비가림감귤/유(대:3kg)', '감귤/진홍/소과/(무:1.5kg)',
       '감귤/비가림/유(소:1.5kg)', '사과 제수용(4개입)', '한라봉/선물용(무:3kg)', '곶감/대봉(10개)',
       '배/선물용(대:7.5kg)', '금귤/유,무(500g)', '사과 선물용

In [ ]:
df_FRUITS_fruits[['물품명', '구매금액']]

,물품명,구매금액
0,사과/유(1.5kg),22207
62,건대추(500g),22973
132,귤/유(1.5kg),11104
187,귤/유(1.5kg),11104
220,배(대:3개),16847
...,...,...
705342,황매실(남고)/유(5kg),-111036
705354,황매실(남고)/유(5kg),111036
705429,냉동산딸기(250g),35608
705467,사과/(2kg)/소,13592


In [ ]:
df_FRUITS_fruits['판매수량'] = df_FRUITS_fruits.groupby('물품명')['물품명'].transform('count')

# 물품명을 기준으로 데이터프레임을 그룹화하고, 각 그룹의 첫 번째 행만 남깁니다.
df_results = df_FRUITS_fruits.drop_duplicates('물품명')
df_results[:2]

<ipython-input-9-95049770b6a6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_FRUITS_fruits['판매수량'] = df_FRUITS_fruits.groupby('물품명')['물품명'].transform('count')


,공급일자,요일,공급월,공급주차,회원번호,조합원상태,물품대분류,물품중분류,물품소분류,물품명,...,주소-구,주소-동,성별,연령,연령대,구매금액,구매매장,반품_원거래일자,구매시각,판매수량
0,2018-01-02,화,1,1,272369856,정상회원,과실,과일,사과,사과/유(1.5kg),...,수지구,풍덕천동,여,45.0,40대,22207,매장C,NaN,10:04,747
62,2018-01-02,화,1,1,445131904,정상회원,과실,과일,건대추,건대추(500g),...,수지구,풍덕천동,여,58.0,50대,22973,매장C,NaN,12:18,44


In [ ]:
import scipy.stats as stats
stats.spearmanr(df_results['판매수량'], df_results['구매금액'])

In [ ]:
round(6.119827037919755e-249)

### 분석 결과
- 통계 결론 : 귀무가설 성립
- 사실 결론 : 수량이 과일 유형의 단가와 관계가 없다. statistic=0.26, pvalue=6.11

In [ ]:
sns.histplot(data=df_FRUITS_fruits, x='물품명', kde=True)
plt.show()

In [ ]:
sns.histplot(data=df_FRUITS_fruits, x='구매금액', kde=True)
plt.show()